# Predicting the Possibilistic Score of OWL Axioms through Support Vector Regression

#### Dario Malchiodi, Célia da Costa Pereira, and Andrea G. B. Tettamanzi

## Code to be used in order to reproduce the experiments

### First step

Execute the cells in this section in order to load the dataset and define the functions to be used in the subsequent experiments.

In [3]:
# Data format:
#
# n0, n1, ...
# mu0, k00, k01, ..., k0n
# ...
# mun, kn0, kn1, ..., knn

import csv
import numpy as np

data_file_name = 'data/data-tettamanzi-complete.csv'

with open(data_file_name) as data_file:
    data = np.array(list(csv.reader(data_file)))

n = len(data) - 1

names = np.array(data[0])[1:n+1]
mu = np.array([float(row[0]) for row in data[1:n+1]])
gram = np.array([[float(k.replace('NA', '0')) for k in row[1:n+1]] for row in data[1:n+1]])

assert(len(names.shape) == 1)
assert(len(mu.shape) == 1)
assert(len(gram.shape) == 2)
assert(names.shape[0] == gram.shape[0] == gram.shape[1] == mu.shape[0])

from possibilearn import *
from possibilearn.kernel import PrecomputedKernel

import csv
import matplotlib.pyplot as plt
import gc

def estimate_possibility(n, mu, cs, gram, num_holdouts, percentages,
                         verbose=False, type='eps-svr'):
    axiom_indices = range(n)
    assert(len(axiom_indices)==len(mu)==n)

    paired_axioms = [axiom_indices[i:i+2] for i in range(0, n, 2)]
    paired_labels = [mu[i:i+2] for i in range(0, n, 2)]

    metrics_membership_rmse = []
    metrics_membership_median = []
    metrics_membership_stdev = []

    metrics_possibility_rmse = []
    metrics_possibility_median = []
    metrics_possibility_stdev = []

    for h in range(num_holdouts):
        (paired_values_train,
         paired_values_validate,
         paired_values_test,
         paired_mu_train,
         paired_mu_validate,
         paired_mu_test) = split_indices(paired_axioms, paired_labels, percentages)

        if verbose:
            print 'holdout {} of {}'.format(h, num_holdouts)
    
        out = model_selection_holdout_reg(paired_values_train, paired_mu_train,
                                                        paired_values_validate, paired_mu_validate,
                                                        cs, gram,
                                                        log=True,
                                                        verbose=verbose,
                                                        type=type)
        if type == 'eps-svr':
            best_c, best_epsilon, result = out
        else:
            best_c, result = out
        
        
        if best_c is None:
            if verbose:
                print 'in holdout {} optimization always failed!'.format(h)
            continue
    
        if verbose:
            if type == 'eps-svr':
                print 'in holdout {} best epsilon is {}'.format(h, best_epsilon)
                
            print 'in holdout {} best C is {}'.format(h, best_c)
        best_model = result[0]
    
        # values and labels are still paired, we need to flatten them out
        values_train = flatten(paired_values_train)
        values_test = flatten(paired_values_test)
        mu_test = flatten(paired_mu_test)
        
        gram_test = [[gram[i, j] for i in values_train] for j in values_test]
        
        mu_test_hat = best_model.predict(gram_test)
        membership_square_err = (mu_test_hat - mu_test)**2
        membership_rmse = math.sqrt(sum(membership_square_err) / len(values_test))
        metrics_membership_rmse.append(membership_rmse)
    
        membership_median = np.median(membership_square_err)
        metrics_membership_median.append(membership_median)
    
        membership_stdev = np.std(membership_square_err)
        metrics_membership_stdev.append(membership_stdev)
        
        estimated_mu = mu_test_hat
        actual_possibility = [mfi - mnotfi for mfi, mnotfi in zip(mu_test[::2], mu_test[1::2])]
        estimated_possibility = [mfi - mnotfi
                                 for mfi, mnotfi in zip(estimated_mu[::2], estimated_mu[1::2])]
    
        possibility_square_err = [(actual - estimated)**2
                              for actual, estimated in zip(actual_possibility, estimated_possibility)]
        possibility_rmse = math.sqrt(sum(possibility_square_err) / len(possibility_square_err))
        metrics_possibility_rmse.append(possibility_rmse)
    
        possibility_median = np.median(possibility_square_err)
        metrics_possibility_median.append(possibility_median)
    
        possibility_stdev = np.std(possibility_square_err)
        metrics_possibility_stdev.append(possibility_stdev)
    
        indices = ['-'.join(map(str, pair)) for pair in paired_values_test]

        results = [(i, phi, notphi, max(phi, notphi), ephi, enotphi, max(ephi, enotphi), p, ep, (p - ep)**2)
                   for i, phi, notphi, p, ephi, enotphi, ep in zip(indices, mu_test[::2], mu_test[1::2], actual_possibility,
                                                            estimated_mu[::2], estimated_mu[1::2], estimated_possibility)]

        results.sort(key = lambda r: r[-1])
    
        with open('data/axioms-reg-{}-results-holdout-{}-details.csv'.format(type, h), 'w') as output_file:
            writer = csv.writer(output_file)
            writer.writerows(results)
    
        with open('data/axioms-reg-{}-results-holdout-{}-global.csv'.format(type, h), 'w') as output_file:
            writer = csv.writer(output_file)
            writer.writerows([
                ('membership RMSE', membership_rmse),
                ('membership median', membership_median),
                ('membership STDEV', membership_stdev),
                ('possibility RMSE', possibility_rmse),
                ('possibility median', possibility_median),
                ('possibility STDEV', possibility_stdev),
            ])
        
        errors = [r[-1] for r in results]
        plt.figure()
        p = plt.boxplot(errors)
        plt.savefig('data/axioms-reg-{}-results-holdout-{}-boxplot.png'.format(type, h))
        plt.clf()
    
        plt.figure()
        p = plt.hist(errors, bins=50)
        plt.savefig('data/axioms-reg-{}-results-holdout-{}-histogram.png'.format(type, h))
        plt.clf()
    
        gc.collect()

    if verbose:
        print 'Membership average values:'
        print 'RMSE: {}'.format(np.average(metrics_membership_rmse))
        print 'Median: {}'.format(np.average(metrics_membership_median))
        print 'STDEV: {}'.format(np.average(metrics_membership_stdev))

        print 'Possibility average values:'
        print 'RMSE: {}'.format(np.average(metrics_possibility_rmse))
        print 'Median: {}'.format(np.average(metrics_possibility_median))
        print 'STDEV: {}'.format(np.average(metrics_possibility_stdev))

    with open('data/axioms-reg-{}-results-holdout-average-metrics.csv'.format(type), 'w') as output_file:
        writer = csv.writer(output_file)
        writer.writerows([
            ('membership average RMSE', np.average(metrics_membership_rmse)),
            ('membership average median', np.average(metrics_membership_median)),
            ('membership average STDEV', np.average(metrics_membership_stdev)),
            ('possibility average RMSE', np.average(metrics_possibility_rmse)),
            ('possibility average median', np.average(metrics_possibility_median)),
            ('possibility average STDEV', np.average(metrics_possibility_stdev)),
        ])

## Second step

Execute the next cell in order to set up the experiments' parameters.

In [4]:
k = PrecomputedKernel(gram)
axiom_indices = range(n)

cs = (10000, 1000, 100, 10, 1, .1, .01, .001)
ks = (k,)

num_holdouts = 10
percentages = (.8, .1, .1)

## Third step: $\epsilon$-insensitive regression

Execute the next cell in order to execute experiments using $\epsilon$-insensitive regression.

In [5]:
estimate_possibility(n, mu, cs, gram, num_holdouts, percentages, verbose=True, type='eps-svr')

holdout 0 of 10
in holdout 0 best epsilon is 100000
in holdout 0 best C is 10000
holdout 1 of 10
in holdout 1 best epsilon is 100000
in holdout 1 best C is 10000
holdout 2 of 10
in holdout 2 best epsilon is 100000
in holdout 2 best C is 10000
holdout 3 of 10
in holdout 3 best epsilon is 100000
in holdout 3 best C is 10000
holdout 4 of 10
in holdout 4 best epsilon is 100000
in holdout 4 best C is 10000
holdout 5 of 10
in holdout 5 best epsilon is 100000
in holdout 5 best C is 10000
holdout 6 of 10
in holdout 6 best epsilon is 100000
in holdout 6 best C is 10000
holdout 7 of 10
in holdout 7 best epsilon is 100000
in holdout 7 best C is 10000
holdout 8 of 10
in holdout 8 best epsilon is 100000
in holdout 8 best C is 10000
holdout 9 of 10
in holdout 9 best epsilon is 100000
in holdout 9 best C is 10000
Membership average values:
RMSE: 0.48261950076
Median: 0.25
STDEV: 0.0393892332903
Possibility average values:
RMSE: 0.845819426046
Median: 0.955068081483
STDEV: 0.410699535238


## Fourth step: ridge regression

Execute the next cell in order to execute experiments using ridge regression.

In [6]:
estimate_possibility(n, mu, cs, gram, num_holdouts, percentages, verbose=True, type='ridge')

holdout 0 of 10


/Users/malchiodi/Applications/anaconda2/lib/python2.7/site-packages/sklearn/linear_model/ridge.py:154: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn("Singular matrix in solving dual problem. Using "


in holdout 0 best C is 10


/Users/malchiodi/Applications/anaconda2/lib/python2.7/site-packages/matplotlib/pyplot.py:516: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


holdout 1 of 10
in holdout 1 best C is 0.001
holdout 2 of 10
in holdout 2 best C is 10
holdout 3 of 10
in holdout 3 best C is 0.01
holdout 4 of 10
in holdout 4 best C is 10000
holdout 5 of 10
in holdout 5 best C is 0.001
holdout 6 of 10
in holdout 6 best C is 10
holdout 7 of 10
in holdout 7 best C is 0.001
holdout 8 of 10
in holdout 8 best C is 0.01
holdout 9 of 10
in holdout 9 best C is 0.1
Membership average values:
RMSE: 0.389328089426
Median: 0.0858485703139
STDEV: 0.266731187893
Possibility average values:
RMSE: 0.628384030353
Median: 0.190587951058
STDEV: 0.637492780973


## Fifth step: collect results of Table 2

Execute the next cell in order to get the LaTeX code for Table 2.

In [8]:
table_2_header = r'''\begin{tabular}{ lrrrrrr }
& \multicolumn{3}{c}{Membership} & \multicolumn{3}{c}{$\mathrm{ARI}$} \\ 
& RMSE & Median & STDEV & RMSE & Median & STDEV \\ 
\hline
'''

table_2_body = r'''$\epsilon$-sensitive & {eps-mu-avg} & {eps-mu-med} & {eps-mu-std} & {eps-ari-avg} & {eps-ari-med} & {eps-ari-std} \\ 
ridge & {rdg-mu-avg} & {rdg-mu-med} & {rdg-mu-std} & {rdg-ari-avg} & {rdg-ari-med} & {rdg-ari-std} \\ 
'''

table_2_footer = r'''\hline
\end{tabular}'''

values_dict = {}

names = ('-mu-avg', '-mu-med', '-mu-std', '-ari-avg', '-ari-med', '-ari-std')

for kind, header in zip(('eps-svr', 'ridge'), ('eps', 'rdg')):
    with open('data/axioms-reg-{}-results-holdout-average-metrics.csv'.format(kind), 'r') as f:
        content = f.read()
        for name, val in zip(names, content.split('\r\n')[:-1]):
            #print '{}{}: {:.2e}'.format(header, name, float(val.split(',')[-1]))
            values_dict['{}{}'.format(header, name)] = '{:.2e}'.format(float(val.split(',')[-1]))

table_2 = table_2_header + '\n' + table_2_body.format(**values_dict) + '\n' + table_2_footer
print table_2

## Sixth step: get histograms in Fig. 1

Histograms in Fig. 1 are picked among those are automatically generated during the third and fourth step and stored in the `data` directory using as name `axioms-reg-<method>-results-holdout-<n>-histogram.png`, where `<method>` is either `ridge` or `eps-svr` and `<n>` is a number between 0 and 9 denoting a given holdout iteration. The directory will also contain boxplots for the same distributions in files having as name `axioms-reg-<method>-results-holdout-<n>-boxplot.png`.

## Seventh step: generate Table 3

Execute the next cell in order to generate the silhouette values when considering both regression methods.

In [ ]:
import pandas as pd

import glob

leading_part = 'data/axioms-reg-'
middle_part = 'results-holdout-'
trailing_part = '-details.csv'

fuzzifiers = set()

for file_name in glob.glob('data/axioms-reg*details.csv'):
    experiment = file_name[len(leading_part):]
    sep_position = experiment.find('-results')
    fuzzifier = experiment[:sep_position]
    fuzzifiers.add(fuzzifier)
    
table = {}

for f in fuzzifiers:

    dfs = []

    for file_name in glob.glob('{}{}-*details.csv'.format(leading_part, f)):
        experiment = file_name[len(leading_part):]
        sep_position= len(f) + len(middle_part)
        num_experiment = experiment[sep_position+1:experiment.find(trailing_part)]
        with open(file_name, 'r') as csv_file:
            data = csv.reader(csv_file)
            results = [r for r in data]
            df = pd.DataFrame(index=[r[0] for r in results])
            df[num_experiment] = [float(r[-1]) for r in results]
            dfs.append(df)

    result = pd.merge(left=dfs[0], right=dfs[1], how='outer', left_index=True, right_index=True)
    for i in range(2, len(dfs)):
        result = pd.merge(left=result, right=dfs[i], how='outer', left_index=True, right_index=True)

    table[f] = result
    table[f]['median'] = table[f].median(numeric_only=True, axis=1)
    table[f]['std'] = table[f].std(numeric_only=True, axis=1)

avg_dfs = [table[f].filter(['median']) for f in fuzzifiers]
fuzzifier_name = [f for f in fuzzifiers]
result = pd.merge(left=avg_dfs[0], right=avg_dfs[1], how='outer',
                  left_index=True, right_index=True)
result.columns = fuzzifier_name[:2]
for i in range(2, len(avg_dfs)):
    result = pd.merge(left=result, right=avg_dfs[i], how='outer',
                      left_index=True, right_index=True)

result.columns = fuzzifier_name

result_filled = result.copy()

m = result_filled.mean(axis=1)
for i, col in enumerate(result_filled):
    result_filled.iloc[:, i] = result_filled.iloc[:, i].fillna(m)
    
from sklearn.cluster import KMeans
from sklearn import metrics

def cluster_and_score(data, k):
    clusterer = KMeans(k)
    clusterization = clusterer.fit(data)
    return (k, metrics.silhouette_score(result_filled,
                                        clusterization.labels_,
                                        metric='euclidean'))
pd.DataFrame([cluster_and_score(result_filled, k) for k in range(2, 10)],
             columns=('$k$', 'Silhouette index'))



Execute the next cell in order to generate the silhouette values when considering only $\epsilon$-insensitive regression.

In [ ]:
eps_dfs = table['eps-svr'].filter(['median'])
eps_dfs.columns = ['eps-svr']

result_filled = eps_dfs.copy()

m = result_filled.mean(axis=1)
for i, col in enumerate(result_filled):
    result_filled.iloc[:, i] = result_filled.iloc[:, i].fillna(m)
    
from sklearn.cluster import KMeans
from sklearn import metrics

def cluster_and_score(data, k):
    clusterer = KMeans(k)
    clusterization = clusterer.fit(data)
    return (k, metrics.silhouette_score(result_filled,
                                        clusterization.labels_,
                                        metric='euclidean'))
pd.DataFrame([cluster_and_score(result_filled, k) for k in range(2, 10)],
             columns=('$k$', 'Silhouette index'))



Execute the next cell in order to generate the silhouette values when considering only ridge regression.

In [ ]:
ridge_dfs = table['ridge'].filter(['median'])
ridge_dfs.columns = ['ridge']

result_filled = ridge_dfs.copy()

m = result_filled.mean(axis=1)
for i, col in enumerate(result_filled):
    result_filled.iloc[:, i] = result_filled.iloc[:, i].fillna(m)
    
from sklearn.cluster import KMeans
from sklearn import metrics

def cluster_and_score(data, k):
    clusterer = KMeans(k)
    clusterization = clusterer.fit(data)
    return (k, metrics.silhouette_score(result_filled,
                                        clusterization.labels_,
                                        metric='euclidean'))
pd.DataFrame([cluster_and_score(result_filled, k) for k in range(2, 10)],
             columns=('$k$', 'Silhouette index'))

Execute the next cell in order to detect easy and hard axioms for the case of ridge regression, and to save their list in the file `hardness-reg-ridge.json`.

In [ ]:
n = len(result_filled)
n = 1444 * 2
avg = np.array([0., 0.])
hardness = [None] * n

for p in result_filled.index:
    cluster_index = clusterization.predict(result_filled.loc[[p]])[0]
    avg[cluster_index] += result_filled.loc[[p]]['ridge'][0]
    for i in map(int, p.split('-')):
        hardness[i] = cluster_index

n_cluster_1 = len([i for i in range(len(hardness)) if hardness[i]])
avg /= (float(n-n_cluster_1)/2, float(n_cluster_1)/2)

easy_index = avg.argmin()
hard_index = 1 - easy_index

easy_axioms = [i for i in range(len(hardness)) if hardness[i]==easy_index]
hard_axioms = [i for i in range(len(hardness)) if hardness[i]==hard_index]

print 'There are {} easy axioms and {} hard axioms.'.format(len(easy_axioms), len(hard_axioms))

import json

with open('hardness-reg-ridge.json', 'w') as f:
    json.dump([1 if hardness[i]==hard_index else 0 for i in range(len(hardness))], f)

Execute the next cell in order to print indices of all hard axioms detected through ridge regression.

In [ ]:
with open('hardness-reg-ridge.json', 'r') as f:
    c = json.load(f)

print 'Hard indices for svr-ridge'
ridge_hard =  [i for i in range(len(c)) if c[i]==1]
print ridge_hard

Execute the next cell in order to obtain the indices of axioms in Table 3 and to print the overlap percentage between hard axioms detected with ridge regression and with fuzzy learn.

In [ ]:
with open('hardness.json', 'r') as f:
    c = json.load(f)

print 'Hard indices for fuzzy-learn'
fuzzy_hard = [i for i in range(len(c)) if c[i]==1]
print fuzzy_hard

print 'common indices between ridge and fuzzy-learn'
common = [i for i in ridge_hard if i in fuzzy_hard]
print common

print 'percentage of overlap'
print float(len(common))/len(ridge_hard)